Dependencies

In [1]:
import random
import os
from glob import glob 

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torchaudio
import torchmetrics as tm
import torchmetrics.classification as tm_cls
import torch

from dataset import VADMelDataModule
from inference import VADPredictor
from utils.load_config import load_config
from utils.get_frame_targets import get_frame_targets
from utils.evaluating import plotly_plot_roc

from IPython.display import Image, display

Constants

In [2]:
EPS = 1e-8
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# np.random.seed(42)

paths = {'sample':'data/samples/5ed8a1c0f3ea2.flac',
        'config_name': '32_n_frames.yml',
        'ckpt_folder': 'checkpoints/32_n_frames',
        'weight_folder': 'weights/VADNet_13_0.1502_0.9506.pt'}

glob_paths = {}

Params

In [3]:
cfg, ckpt_folder = load_config('./configs/' + paths['config_name'])
params = {key: value for key, value in cfg['data'].items()}

Model

In [4]:
predictor = VADPredictor(ckpt_folder = paths['ckpt_folder'], weight_folder = paths['weight_folder'], device=device)
mel_spec =  torchaudio.transforms.MelSpectrogram(n_fft=params['nfft'], hop_length=params['hop_length'], n_mels=params['n_mels'])

### ROC - curve. AUROC.

For a sample.

In [5]:
audio, _ = torchaudio.load(paths['sample'])
spec = torch.log(mel_spec(audio) + EPS)
total_frames = spec.shape[-1]
probs = torch.from_numpy(predictor.predict(audio_path = paths['sample'], threshold = None))
label = get_frame_targets(paths['sample'], total_frames=total_frames, hop_length=params['hop_length']).squeeze(0).to(torch.long)

In [6]:
# roc
bn_roc = tm_cls.BinaryROC(thresholds = None).to(device)
fprs_i, tprs_i, thresholds_i = bn_roc(preds = probs, target = label)
# auc
bn_auroc = tm_cls.BinaryAUROC(thresholds = None).to(device)
auc_value = bn_auroc(preds = probs, target = label).item()
print('AUC: ', auc_value)

AUC:  0.9997934103012085


If you need plot, uncomment

In [1]:
# plots
# plotly_plot_roc(fprs_i, tprs_i, auc_value, desire_fr_val_i=0.0, desire_tr_val_i=0.99, vertical_line = False)
# do not plot to upload to git
# display(Image(filename='pics/roc_curve_for_sample.png'))

In [8]:
desire_fr_val_i = 0.0
desire_tr_val_i = 0.99

In [9]:
threshold_index_i = np.argmin(np.abs(fprs_i - desire_fr_val_i) + np.abs(tprs_i - desire_tr_val_i))
print('Threshold index for desired fpr and tpr:', threshold_index_i.item())
print('Threshold:', thresholds_i[threshold_index_i].item()) 

Threshold index for desired fpr and tpr: 227
Threshold: 0.6504456996917725


**Inferencing**

In [10]:
# Path to sample: data/samples/5ed8a1c0f3ea2.flac
# Threshold: 0.87
# Path to checkpoints: checkpoints/current_run
# Path to weights: weights/VADNet_13_0.1502_0.9506.pt   
!python inference.py data/samples/5ed8a1c0f3ea2.flac -plot -s -t 0.65 -c checkpoints/32_n_frames -w weights/VADNet_13_0.1502_0.9506.pt  

If you need plot, uncomment

In [2]:
# display(Image(filename='pics/result.png'))

In [11]:
# resets
bn_roc.reset()
bn_auroc.reset()

### For samples.

In [12]:
samples = glob(os.path.join(str(params['data_dir']), '*.flac'))

In [13]:
bn_roc = tm_cls.BinaryROC(thresholds = None).to(device)
bn_auroc = tm_cls.BinaryAUROC(thresholds = None).to(device)

for t in tqdm(samples):
    t = t.replace('\\', '/')
    audio, _ = torchaudio.load(t)
    spec = torch.log(mel_spec(audio) + EPS)
    total_frames = spec.shape[-1]
    pred = torch.from_numpy(predictor.predict(audio_path = t, threshold = None))
    target = get_frame_targets(t, total_frames=total_frames, hop_length=params['hop_length']).squeeze(0).to(torch.long)
    with torch.no_grad():
        bn_roc.update(preds=pred, target=target) # roc
        bn_auroc.update(preds=pred, target=target) # auc

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 20388/20388 [02:29<00:00, 136.19it/s]


In [14]:
auc_value = bn_auroc.compute().item()
print('AUC: ', auc_value) # auc

AUC:  0.9754916429519653


In [ ]:
fprs, tprs, thresholds = bn_roc.compute() # roc

If you need plot, uncomment

In [3]:
# plots
# plotly_plot_roc(fprs, tprs, auc_value, desire_fr_val_i=0.08, desire_tr_val_i=0.93, vertical_line=True) 
# do not plot to upload to git
# display(Image(filename='pics/roc-curve-for-samples.png'))

In [24]:
desire_fr_val = 0.08
desire_tr_val = 0.93
threshold_index = np.argmin(np.abs(fprs - desire_fr_val) + np.abs(tprs - desire_tr_val))
print('Threshold index for desired fpr and tpr:', threshold_index.item())
print('Threshold:', thresholds[threshold_index].item())

Threshold index for desired fpr and tpr: 706552
Threshold: 0.665117084980011


In [26]:
# resets
bn_roc.reset()
bn_auroc.reset()

### Metrics.

In [5]:
threshold = 0.65 
params['n_workers'] = 10

In [6]:
datamodule = VADMelDataModule(**params).setup(info = False)
test_dataloader = datamodule.test_dataloader()

In [7]:
metrics = {m: getattr(tm_cls, m)(threshold = threshold, validate_args=False).to(device) for m in ['BinaryAccuracy', 'BinaryPrecision', 'BinaryRecall', 'BinaryF1Score']}

In [8]:
for inputs, labels in tqdm(test_dataloader):
    inputs, targets = inputs.to(device), labels.to(device)
    with torch.no_grad():
        preds = predictor.model(inputs).transpose(1, 2)    
    [metrics[m].update(preds=preds, target=targets) for m in metrics.keys()]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:29<00:00,  7.49s/it]


In [9]:
computed_metrics = {m: metrics[m].compute().item() for m in metrics.keys()}
_ = [metrics[m].reset() for m in metrics.keys()]

In [10]:
_ = [print(f"{m}: {computed_metrics[m]:.4f} ", end="") for m in metrics.keys()]

BinaryAccuracy: 0.9550 BinaryPrecision: 0.9615 BinaryRecall: 0.9444 BinaryF1Score: 0.9529 

### Comparing: MarbleNet vs VadNet

In [ ]:
# Todo